#Sentiment Analysis using TextBlob

###Importing Libraries

In [2]:
import re
import pandas as pd
import ast
import json
from nltk import tokenize
from textblob import TextBlob

###Defining Split into Phrases function

In [29]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

###Reading Dataset

In [24]:
ratemd=pd.read_csv(r'/Users/sangh/Desktop/Final Project/Healthgrades/All_Healthgrades_v3.csv')
ratemd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 8 columns):
DoctorName             840 non-null object
Age                    839 non-null float64
Affiliated Hospital    839 non-null object
Gender                 840 non-null object
Education              834 non-null object
Experience             840 non-null int64
Rating                 840 non-null float64
Reviews                840 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 52.6+ KB


In [214]:
#Concatenating all reviews for each physician
ratemd["Single_review"] = ratemd["Doctor_reviews"].to_list()

In [309]:
sentrm=[]
doc_names = []
star_rating = []
gender = []
hos_affil = []
nr1,nc1=ratemd.shape

for j in range(nr1):
    name = ratemd.iloc[j,0]
    t1=ratemd.iloc[j,7]
    t1 = t1.strip("][").split(",")
    for x in range(len(t1)):
        x_sent = t1[x]
        sentrm.append(x_sent)
        doc_names.append(name)       

In [310]:
len(sentrm)

10424

In [311]:
de = {"Doc_names":doc_names, "Review_phrases":sentrm}
df = pd.DataFrame(de)

In [313]:
df.shape

(10424, 2)

In [169]:
ratemd.to_excel(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Ratemds - Illinois/New/complete_data_sentences_withbut.xlsx',header=False)

In [69]:
#pip install vaderSentiment
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#analyser1 = SentimentIntensityAnalyzer()
#def sentiment_analyzer_scores(sentence):
#    score = analyser1.polarity_scores(sentence)
    #print("{:-<40} {}".format(sentence, str(score)))
#    print(score)
#    if score['compound']>0.05:
#        return "Positive"   
#    elif score['compound']<-0.05:       
#        return "Negative"    
#    else:       
#        return "Neutral"

In [71]:
txt = "he not only loves to bring new life into the world but dr. drachler is seriously concerned about the quality of our lives"
sentiment_analyzer_scores(txt)

{'neg': 0.162, 'neu': 0.838, 'pos': 0.0, 'compound': -0.4676}


'Negative'

### Defining Sentiment Analyser function using TextBlob

In [25]:
#Function to determine the sentiment of the sentence
i=[]
j=[]
k=[]

def sentiment_analyser(text):
    analysis = TextBlob(text)
    if analysis.sentiment[0]>0:
        i.append(1)
        #return "Positive"   
        return analysis.sentiment[0]
    elif analysis.sentiment[0]<0:       
        j.append(1)
        #return "Negative"    
        return analysis.sentiment[0]
    else:       
        k.append(1)
        #return "Neutral"
        return analysis.sentiment[0]

In [177]:
nr1,nc1=df.shape

In [67]:
#function to take all the sentences, identify sentiment and provide a dictionary with key as sentence and value as sentiment
def sentence_sentiment(df):
    main_list = []
    nr1,nc1 = df.shape
    for j in range(nr1):
        t1=ratemd.iloc[j,2]
        sub_list = []
        for x in range(len(t1)):
            d = {}
            senti = sentiment_analyser(t1[x])
            d[t1[x]] = senti
            for f in d.copy():
                if d[f] == 'Neutral':
                    d.pop(f)
            sub_list.append(d)
        main_list.append(sub_list)
    return main_list

In [28]:
def sentence_sentiment1(df):
    main_list = []
    nr1,nc1 = df.shape
    for j in range(nr1):
        t1=df.iloc[j,7]
        senti = sentiment_analyser(t1)
        main_list.append(senti)
    return main_list

In [29]:
sentiment = sentence_sentiment1(ratemd)

In [30]:
len(sentiment)

840

In [129]:
for x in range(len(sentiment)): 
    q = sentiment[x]
    for a in range(len(q)):
        sent = q[a]
        for f in sent.keys():
            if sent[f] == 'Neutral':
                del(sent[f])

In [131]:
for x in range(len(sentiment)): 
    q = sentiment[x]
    for a in range(len(q)):
        sent = q[a]
        for f in sent.copy():
            if not sent[f]:
                sent.pop(f)

In [156]:
filter(None, sentiment)

In [31]:
len(sentiment)
print(len(i))
print(len(j))
print(len(k))

832
254
270


In [32]:
ratemd['Whole_Sentiment']=sentiment

In [122]:
df = df[df.Sentiment != "Neutral"]

In [320]:
sent = []
for x in range(len(df["Sentiment"])):
    if df["Sentiment"][x] > 0:
        sent.append("Positive")
    elif df["Sentiment"][x] < 0:
        sent.append("Negative")
    else:
        sent.append("Neutral")
    

In [250]:
len(sent)

6621

In [321]:
df["sent"] = sent

In [126]:
df_negative.to_csv(r'/Users/sangh/Desktop/Final Project/RateMds/Phrases_Negative.csv', header=True, index=False)
df_positive.to_csv(r'/Users/sangha/Desktop/Final Project/RateMds/Phrases_Positive.csv', header=True, index=False)